# Pre-Elaborazione dei Dati (Dataset di riferimento 2025 da Gennaio a Giugno)

In [1]:
import pandas as pd
import glob
import os
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import shutil
import re 

### Delineamo l'ambiente di lavoro

In questa sezione vengono definite le directory di lavoro e tutti quei parametri per cui andiamo a filtrare i nostri dati.

SOG_MIN --> Impostiamo il parametro a 2.0, questo ci serve per poi andare a scartare tutte le navi ferme.

TIME_GAP --> Questa è una soglia di tempo massima arbitraria permessa all'interno di una singola traiettoria. Se tra due messaggi consecutivi della stessa nave passano più di 60 minuti, assumiamo che la rotta sia stata interrotta.

In [3]:
INPUT_DIR = '../../../Dataset'
SCRIPT_DIR = os.getcwd()                                # Restituisce la directory di lavoro corrente

OUTPUT_DIR_NAME = 'Dataset_Pre-Cleaned_AIS' 
OUTPUT_DIR = os.path.join(SCRIPT_DIR, OUTPUT_DIR_NAME)

SOG_MIN_THRESHOLD = 2.0
TIME_GAP_THRESHOLD = pd.Timedelta(hours=1)

os.makedirs(OUTPUT_DIR, exist_ok=True)

all_files = glob.glob(os.path.join(INPUT_DIR, '*.parquet'))

all_clean_data = []

print(f"Trovati {len(all_files)} file Parquet da processare.")

Trovati 365 file Parquet da processare.


#### Test

- Proviamo a verificare la lettura di un file parquet e della corretta formattazione dei dati.  
- Oltre a questo andiamo ad estrarre il numero di colonne per verificare se sono state selezionate le colonne corrette.  
- Viene aggiunto anche un controllo sulle righe per vedere dopo la pulizia la percentuale di pulizia per ogni file.



In [4]:
BASE_PATH = '../../../Dataset/'

FILE_PATH_TEST = os.path.join(BASE_PATH, 'ais-2025-01-01.parquet')
FILE_PATH_TEST2 = os.path.join(BASE_PATH, 'AIS_2024_12_31.parquet')

COLUMN_MAPPING2025 = {
    'mmsi': 'MMSI', 
    'latitude': 'Latitude', 
    'longitude': 'Longitude', 
    'sog': 'SOG', 
    'cog': 'COG', 
    'base_date_time': 'Timestamp' 
}
COLUMNS_TO_READ_2025 = list(COLUMN_MAPPING2025.keys())

try:
    df = pd.read_parquet(
        FILE_PATH_TEST, 
        columns=COLUMNS_TO_READ_2025,
        engine='pyarrow' 
    )

    df = df.rename(columns=COLUMN_MAPPING2025)

    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    
    
    print(f"--- 🔍 DEBUG: Dati iniziali dal file {os.path.basename(FILE_PATH_TEST)} ---")
    print("\nHead del DataFrame:")
    print(df.head())
    print("\nTipi di Dati (Dtypes) dopo la conversione Timestamp:")
    print(df.dtypes)
    print("----------------------------------------------------------------\n")
    rows,columns = df.shape
    print(f"Numero di righe: {rows}, Numero di colonne: {columns}\n")
        
except Exception as e:
    print(f"Errore nel processare il file {FILE_PATH_TEST}: {e}")


COLUMN_MAPPING2024 = {
    'MMSI': 'MMSI',
    'LAT': 'Latitude',
    'LON': 'Longitude',
    'SOG': 'SOG',
    'COG': 'COG',
    'BaseDateTime': 'Timestamp' 
}
COLUMNS_TO_READ_2024 = list(COLUMN_MAPPING2024.keys())

try:
    df = pd.read_parquet(
        FILE_PATH_TEST2, 
        columns=COLUMNS_TO_READ_2024,
        engine='pyarrow' 
    )

    df = df.rename(columns=COLUMN_MAPPING2024)

    df['Timestamp'] = pd.to_datetime(df['Timestamp'])


    print(f"--- 🔍 DEBUG: Dati iniziali dal file {os.path.basename(FILE_PATH_TEST2)} ---")
    print("\nHead del DataFrame:")
    print(df.head())
    print("\nTipi di Dati (Dtypes) dopo la conversione Timestamp:")
    print(df.dtypes)
    print("----------------------------------------------------------------\n")
    rows,columns = df.shape
    print(f"Numero di righe: {rows}, Numero di colonne: {columns}\n")
        
except Exception as e:
    print(f"Errore nel processare il file {FILE_PATH_TEST2}: {e}")

--- 🔍 DEBUG: Dati iniziali dal file ais-2025-01-01.parquet ---

Head del DataFrame:
        MMSI  Latitude  Longitude  SOG    COG           Timestamp
0  671087100  18.46281  -66.10297  0.0  176.7 2025-01-01 00:00:00
1  367733950  48.48503 -122.60927  0.0  215.5 2025-01-01 00:00:00
2  368138010  40.47715  -73.84652  5.5  286.9 2025-01-01 00:00:02
3  367637210  29.12033  -90.21215  0.0  227.6 2025-01-01 00:00:03
4  368050000  41.27196  -72.46934  0.0  107.1 2025-01-01 00:00:03

Tipi di Dati (Dtypes) dopo la conversione Timestamp:
MMSI                  int64
Latitude            float64
Longitude           float64
SOG                 float64
COG                 float64
Timestamp    datetime64[ns]
dtype: object
----------------------------------------------------------------

Numero di righe: 7337208, Numero di colonne: 6

--- 🔍 DEBUG: Dati iniziali dal file AIS_2024_12_31.parquet ---

Head del DataFrame:
        MMSI  Latitude  Longitude   SOG    COG           Timestamp
0  367776660  21.19

### Pulizia dei dati
 
In questa sezione, iteriamo su ogni file del nostro dataset ed eseguiamo la pulizia vera e propria, applicando dei filtri. Il primo filtro filtro applicato è sulla lettura delle colonne `COLUMNS_TO_READ` prima di caricare i dati. È il modo più efficiente per scartare le colonne inutili e riduce drasticamente l'utilizzo della RAM velocizzando l'intero processo.

##### Filtri Navigazione Attiva e di Validità
  
Vengono applicati una serie di filtri per lasciare all'interno del dataset solo valori validi e di navigazione attiva:
1. Applichiamo il filtro `df = df[df['SOG'] > SOG_MIN_THRESHOLD`, eliminando i dati statici come deciso sopra.
2. Applichiamo il filtro `df[df['COG'] != 511]`,rimuovendo i record dove il COG (Course Over Ground) è $511$. Questo è un codice standard AIS che significa "Dato Non Disponibile". Senza una rotta (COG), l'informazione cinematica è incompleta e inutile per il modello.
3. Applichiamo il filtro `Filtro Lat/Lon (>= -90, <= 90, etc.)`, eliminiamo i record con coordinate geografiche errate (fuori dal globo). Questi sono errori di trasmissione o del sensore che inquinerebbero il dataset.
4. Utilizziamo il metodo `df.dopna(...)` per rimuovere qualsiasi riga che abbia valori mancanti. Questo perchè i modelli LSTM/LNN richiedono input completi per funzionare correttamente.
5. Infine l'ultimo filtro è `df['MMSI'].str.len()==9` per rimuovere i record con l'identificativo della nave non corretto. Questo perchè l'MMSI deve essere di 9 cifre e questo ci garantisce che ogni traiettoria sia attribuita ad una nave valida.

##### Segmentazione e Creazione delle Traiettorie

Questa è la fase finale prima del salvataggio, dove trasformiamo i dati puliti in sequenze coerenti (TrajectoryID).  
Quello che andiamo a fare è raggruppare i nostri dati prima per l'MMSI e poi per il TimeStamp. In questo modo abbiamo i dati ordinati ed  è possibile delineare quelle che sono le tratiettorie diverse per ogni nave. Viene aggiunta una nuova colonna al dataset che è `TrajectoryID` che ha il compito di raggruppare tutti i dati di ogni singola nave che fanno riferimento ad un intero spostamento.  
Gli spostamenti sono stati delineati assumendo che spostamenti diversi vengono caratterizzati da uno stato di navigazione non attiva di almeno 1 ora.  
Questa fase è essenziale perchè i modelli che andremo ad addestrare, impareranno non dai singoli punti ma dalle intere sequenze.

```
df = df.sort_values(by=['MMSI', 'Timestamp']).reset_index(drop=True)

            df['TimeDiff'] = df.groupby('MMSI')['Timestamp'].diff()
            df['IsNewTraj'] = (df['MMSI'] != df['MMSI'].shift(1)) | (df['TimeDiff'] > TIME_GAP_THRESHOLD)
            df['TrajectoryID'] = df['IsNewTraj'].cumsum()
```


L'ultima parte serve solo per salvare i file .parquet puliti e segmentati nel dataset che poi servirà per addestrare i modelli.

In [5]:
MAPPING_2025 = {
    'mmsi': 'MMSI', 
    'latitude': 'Latitude', 
    'longitude': 'Longitude', 
    'sog': 'SOG', 
    'cog': 'COG', 
    'base_date_time': 'Timestamp' 
}

COLUMNS_2025 = list(MAPPING_2025.keys())

MAPPING_2024 = {
    'MMSI': 'MMSI',
    'LAT': 'Latitude',
    'LON': 'Longitude',
    'SOG': 'SOG',
    'COG': 'COG',
    'BaseDateTime': 'Timestamp'
}
COLUMNS_2024 = list(MAPPING_2024.keys())

for file_path in all_files:
    df = None
    mapping_usato = None

    try:
        
        df = pd.read_parquet(
            file_path, 
            columns=COLUMNS_2025,
            engine='pyarrow' 
        )
        df = df.rename(columns=MAPPING_2025)
        mapping_usato = "2025"
    
    except Exception as e1:
        try:
            df = pd.read_parquet(
                file_path, 
                columns=COLUMNS_2024,
                engine='pyarrow' 
            )
            df = df.rename(columns=MAPPING_2024)
            mapping_usato = "2024"
        
        except Exception as e2:
            print(f"Errore IRRISOLVIBILE nel caricare {file_path}: Schema non riconosciuto.")
            continue

    if df is not None:
        try:
            
            df['Timestamp'] = pd.to_datetime(df['Timestamp'])
            
            # Filtri cinematici e geografici
            df = df[df['SOG'] > SOG_MIN_THRESHOLD]
            df = df[df['COG'] != 511]
            df = df[(df['Latitude'] >= -90) & (df['Latitude'] <= 90)]
            df = df[(df['Longitude'] >= -180) & (df['Longitude'] <= 180)]
            
            # Filtri di integrità
            df = df.dropna(subset=['MMSI', 'Latitude', 'Longitude', 'SOG', 'COG'])
            df['MMSI'] = df['MMSI'].astype(str).str.replace(r'\D', '', regex=True)
            df = df[df['MMSI'].str.len() == 9]

            if not df.empty:

                output_filename = os.path.basename(file_path).lower()
                output_file = os.path.join(OUTPUT_DIR, output_filename)
                
                df.to_parquet(output_file, index=False)
                
                print(f"File {os.path.basename(file_path)} pre-pulito")
            
        except Exception as e:
            print(f"Errore nella FASE DI PULIZIA per il file {file_path}: {e}")

print("\n--- FASE 1 (Pre-Pulizia) completata. ---")

File ais-2025-02-15.parquet pre-pulito
File ais-2025-06-22.parquet pre-pulito
File ais-2025-04-26.parquet pre-pulito
File AIS_2024_08_21.parquet pre-pulito
File ais-2025-01-22.parquet pre-pulito
File ais-2025-02-27.parquet pre-pulito
File ais-2025-06-18.parquet pre-pulito
File ais-2025-06-14.parquet pre-pulito
File AIS_2024_08_11.parquet pre-pulito
File AIS_2024_08_06.parquet pre-pulito
File AIS_2024_08_22.parquet pre-pulito
File ais-2025-06-21.parquet pre-pulito
File AIS_2024_11_05.parquet pre-pulito
File AIS_2024_10_30.parquet pre-pulito
File AIS_2024_10_16.parquet pre-pulito
File ais-2025-01-23.parquet pre-pulito
File ais-2025-03-26.parquet pre-pulito
File AIS_2024_09_24.parquet pre-pulito
File ais-2025-03-25.parquet pre-pulito
File AIS_2024_07_01.parquet pre-pulito
File AIS_2024_09_11.parquet pre-pulito
File ais-2025-03-08.parquet pre-pulito
File AIS_2024_11_21.parquet pre-pulito
File ais-2025-04-25.parquet pre-pulito
File AIS_2024_08_18.parquet pre-pulito
File AIS_2024_12_19.parqu

#### Test file pre-pulizia

In [5]:
PRE_CLEANED_FILE_PATH_TEST = 'Dataset_Pre-Cleaned_AIS/ais-2025-01-01.parquet'
COLUMNS_TO_READ_2025 = ['MMSI', 'Latitude', 'Longitude','SOG', 'COG', 'Timestamp']

df = pd.read_parquet(
        PRE_CLEANED_FILE_PATH_TEST, 
        columns=COLUMNS_TO_READ_2025,
        engine='pyarrow' 
    )

df.head()

,MMSI,Latitude,Longitude,SOG,COG,Timestamp
0,368138010,40.47715,-73.84652,5.5,286.9,2025-01-01 00:00:02
1,367188610,27.93936,-82.45703,2.2,147.6,2025-01-01 00:00:04
2,366938780,46.04232,-83.93567,11.8,126.0,2025-01-01 00:00:00
3,316028554,49.28782,-123.10689,7.8,215.6,2025-01-01 00:00:06
4,338122081,37.78262,-122.38452,3.7,196.6,2025-01-01 00:00:12


#### Unificazione in un unico file

In [7]:
import pandas as pd
import glob
import os
import numpy as np
import shutil
import math # Per dividere in blocchi

# --- 1. CONFIGURAZIONE ---
INPUT_DIR = 'Dataset_Pre-Cleaned_AIS' 
SCRIPT_DIR = os.getcwd()

# ⭐️ Output per i blocchi da 15 giorni
OUTPUT_DIR_NAME = 'Dataset_Segmentato_15Giorni' 
OUTPUT_DIR = os.path.join(SCRIPT_DIR, OUTPUT_DIR_NAME)

if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)
os.makedirs(OUTPUT_DIR, exist_ok=True)

TIME_GAP_THRESHOLD = pd.Timedelta(hours=1)
# ⭐️ MODIFICA CHIAVE: Come hai suggerito!
GIORNI_PER_BLOCCO = 15

# --- 2. TROVA E RAGGRUPPA FILE PER BLOCCHI ---
print(f"Ricerca file pre-puliti in: {INPUT_DIR}")
all_files = glob.glob(os.path.join(INPUT_DIR, '*.parquet'))
all_files.sort() # Fondamentale per ordinare i giorni!

num_blocchi = math.ceil(len(all_files) / GIORNI_PER_BLOCCO)
print(f"Trovati {len(all_files)} file, raggruppati in {num_blocchi} blocchi da 15 giorni.")

# --- 3. LOOP DI ELABORAZIONE "BLOCCHI 15 GIORNI" (SOLO PANDAS) ---

max_trajectory_id_globale = 0 

for i in range(num_blocchi):
    start_index = i * GIORNI_PER_BLOCCO
    end_index = (i + 1) * GIORNI_PER_BLOCCO
    
    file_list_blocco = all_files[start_index:end_index]
    
    print(f"\n--- Inizio elaborazione Blocco {i+1}/{num_blocchi} ---")
    
    try:
        # --- FASE A: Carica il blocco in RAM ---
        print(f"Caricamento di {len(file_list_blocco)} file...")
        
        df_list = [pd.read_parquet(f) for f in file_list_blocco]
        df_blocco = pd.concat(df_list, ignore_index=True)
        
        print(f"Blocco caricato. Righe: {len(df_blocco)}. Memoria: {df_blocco.memory_usage(deep=True).sum() / (1024**3):.2f} GB")

        # --- FASE B: Segmentazione (in RAM) ---
        print("Inizio ordinamento...")
        df_blocco = df_blocco.sort_values(by=['MMSI', 'Timestamp']).reset_index(drop=True)
        
        print("Inizio calcolo TimeDiff...")
        df_blocco['TimeDiff'] = df_blocco.groupby('MMSI')['Timestamp'].diff()
        
        print("Inizio calcolo IsNewTraj...")
        df_blocco['IsNewTraj'] = (df_blocco['MMSI'] != df_blocco['MMSI'].shift(1)) | (df_blocco['TimeDiff'] > TIME_GAP_THRESHOLD)
        
        print("Inizio calcolo TrajectoryID...")
        # ⭐️ CORREZIONE DEL BUG 'bool + int' ⭐️
        df_blocco['IsNewTraj_int'] = df_blocco['IsNewTraj'].astype(int)
        df_blocco['TrajectoryID'] = df_blocco['IsNewTraj_int'].cumsum() + max_trajectory_id_globale
        
        # --- FASE C: Salvataggio ---
        print("Salvataggio in corso...")
        df_blocco = df_blocco.drop(columns=['TimeDiff', 'IsNewTraj', 'IsNewTraj_int'])
        
        max_trajectory_id_globale = df_blocco['TrajectoryID'].max()
        
        output_file = os.path.join(OUTPUT_DIR, f"blocco_{i:03d}-segmentato.parquet")
        df_blocco.to_parquet(output_file, index=False, engine='pyarrow', compression='snappy')
        
        print(f"--- ✅ Blocco {i+1} salvato! (Max ID: {max_trajectory_id_globale}) ---")

    except MemoryError:
        print(f"--- ❌ ERRORE DI MEMORIA: Blocco {i+1} (15 giorni) è ancora troppo grande! ---")
        print("Riprova con GIORNI_PER_BLOCCO = 7.")
        break 
    except Exception as e:
        print(f"--- ❌ ERRORE SCONOSCIUTO nel blocco {i+1}: {e} ---")

print("\n\n--- 🎉 FASE A (Blocchi 15 Giorni) completata! ---")
print(f"I file intermedi sono in: {OUTPUT_DIR}")
print("\nOra sei pronto per la FASE B (Stitching).")

Ricerca file pre-puliti in: Dataset_Pre-Cleaned_AIS
Trovati 365 file, raggruppati in 25 blocchi da 15 giorni.

--- Inizio elaborazione Blocco 1/25 ---
Caricamento di 15 file...
Blocco caricato. Righe: 39966991. Memoria: 3.65 GB
Inizio ordinamento...
Inizio calcolo TimeDiff...
Inizio calcolo IsNewTraj...
Inizio calcolo TrajectoryID...
Salvataggio in corso...
--- ✅ Blocco 1 salvato! (Max ID: 333169) ---

--- Inizio elaborazione Blocco 2/25 ---
Caricamento di 15 file...
Blocco caricato. Righe: 40481433. Memoria: 3.69 GB
Inizio ordinamento...
Inizio calcolo TimeDiff...
Inizio calcolo IsNewTraj...
Inizio calcolo TrajectoryID...
Salvataggio in corso...
--- ✅ Blocco 2 salvato! (Max ID: 672634) ---

--- Inizio elaborazione Blocco 3/25 ---
Caricamento di 15 file...
Blocco caricato. Righe: 41344475. Memoria: 3.77 GB
Inizio ordinamento...
Inizio calcolo TimeDiff...
Inizio calcolo IsNewTraj...
Inizio calcolo TrajectoryID...
Salvataggio in corso...
--- ✅ Blocco 3 salvato! (Max ID: 1011871) ---

---

In [4]:
import pandas as pd
import glob
import os
import numpy as np
import shutil
import gc

# --- 1. CONFIGURAZIONE ---
INPUT_DIR = 'Dataset_Segmentato_15Giorni' 
OUTPUT_DIR_NAME = 'Dataset_Stitched_Finale' 
OUTPUT_DIR = os.path.join(os.getcwd(), OUTPUT_DIR_NAME)

if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)
os.makedirs(OUTPUT_DIR, exist_ok=True)

TIME_GAP_THRESHOLD = pd.Timedelta(hours=1)

print(f"--- FASE B: Stitching Sequenziale avviato ---")
print(f"Lettura blocchi da: {INPUT_DIR}")
print(f"Salvataggio finale in: {OUTPUT_DIR}\n")

all_blocks = sorted(glob.glob(os.path.join(INPUT_DIR, '*.parquet')))

if not all_blocks:
    print(f"ERRORE: Nessun file blocco trovato in {INPUT_DIR}.")
else:
    # --- PASSAGGIO 1: Copia il primo blocco ---
    # Il primo blocco (blocco_000) non ha correzioni, è già perfetto.
    # Lo copiamo e lo usiamo come punto di partenza.
    
    path_A_corretto = all_blocks[0]
    output_path_A = os.path.join(OUTPUT_DIR, os.path.basename(path_A_corretto))
    shutil.copy(path_A_corretto, output_path_A)
    print(f"Blocco 0 ({os.path.basename(path_A_corretto)}) copiato, nessuna correzione necessaria.")

    # --- PASSAGGIO 2: Loop di aggiornamento sequenziale ---
    # path_A_corretto ora punta al file *già salvato* e corretto nella cartella di output
    
    for i in range(len(all_blocks) - 1):
        # A = Blocco N (già corretto, nella cartella OUTPUT)
        # B = Blocco N+1 (grezzo, dalla cartella INPUT)
        
        path_A_corretto = os.path.join(OUTPUT_DIR, os.path.basename(all_blocks[i]))
        path_B_grezzo = all_blocks[i+1]
        
        print(f"\n--- Inizio cucitura: {os.path.basename(path_A_corretto)} -> {os.path.basename(path_B_grezzo)} ---")
        
        try:
            # 1. Carica A (corretto) e B (grezzo)
            df_A = pd.read_parquet(path_A_corretto)
            df_B = pd.read_parquet(path_B_grezzo)
            
            # 2. Trova la mappa (A_corretto -> B_grezzo)
            print("  Trovati confini, calcolo mappa...")
            last_records_A = df_A.loc[df_A.groupby('MMSI')['Timestamp'].idxmax()]
            last_records_A = last_records_A[['MMSI', 'Timestamp', 'TrajectoryID']].rename(
                columns={'Timestamp': 'Last_Timestamp', 'TrajectoryID': 'Correct_ID'}
            )

            first_records_B = df_B.loc[df_B.groupby('MMSI')['Timestamp'].idxmin()]
            first_records_B = first_records_B[['MMSI', 'Timestamp', 'TrajectoryID']].rename(
                columns={'Timestamp': 'First_Timestamp', 'TrajectoryID': 'Old_ID'}
            )

            boundary_check = pd.merge(last_records_A, first_records_B, on='MMSI')
            boundary_check['TimeDiff'] = boundary_check['First_Timestamp'] - boundary_check['Last_Timestamp']
            stitch_candidates = boundary_check[boundary_check['TimeDiff'] <= TIME_GAP_THRESHOLD]
            
            # Mappa di correzione: {ID_sbagliato_in_B -> ID_corretto_di_A}
            local_fix_map = stitch_candidates.set_index('Old_ID')['Correct_ID'].to_dict()
            print(f"  -> Trovate {len(local_fix_map)} cuciture da applicare.")

            # 3. Libera A dalla memoria (come hai suggerito tu)
            print("  Rilascio memoria Blocco A...")
            del df_A, last_records_A, first_records_B, boundary_check, stitch_candidates
            gc.collect()

            # 4. Applica la mappa a B (Memoria: B + B_copia)
            print("  Applicazione correzioni a Blocco B...")
            # map() è più veloce di replace() per questo compito
            df_B['TrajectoryID'] = df_B['TrajectoryID'].map(local_fix_map).fillna(df_B['TrajectoryID']).astype(int)

            # 5. Salva B corretto
            output_path_B = os.path.join(OUTPUT_DIR, os.path.basename(path_B_grezzo))
            df_B.to_parquet(output_path_B, index=False, engine='pyarrow', compression='snappy')
            print(f"  -> Blocco {os.path.basename(output_path_B)} corretto e salvato.")

        except Exception as e:
            print(f"  -> ❌ ERRORE durante la cucitura: {e}")
            break # Interrompi se qualcosa va storto
        finally:
            # Pulizia finale del loop
            if 'df_A' in locals(): del df_A
            if 'df_B' in locals(): del df_B
            gc.collect()

    print("\n\n--- 🎉 FASE B (Stitching Sequenziale) completata! ---")
    print(f"Dataset perfetto e cucito salvato in: {OUTPUT_DIR}")

--- FASE B: Stitching Sequenziale avviato ---
Lettura blocchi da: Dataset_Segmentato_15Giorni
Salvataggio finale in: /home/al3th3ia/Scrivania/Cybersecurity/Detecting-Trajectory-Spoofing-Attacks-on-AIS/Progetto/Pre-Elaborazione Dati/Dataset_Stitched_Finale

Blocco 0 (blocco_000-segmentato.parquet) copiato, nessuna correzione necessaria.

--- Inizio cucitura: blocco_000-segmentato.parquet -> blocco_001-segmentato.parquet ---
  Trovati confini, calcolo mappa...
  -> Trovate 3127 cuciture da applicare.
  Rilascio memoria Blocco A...
  Applicazione correzioni a Blocco B...
  -> Blocco blocco_001-segmentato.parquet corretto e salvato.

--- Inizio cucitura: blocco_001-segmentato.parquet -> blocco_002-segmentato.parquet ---
  Trovati confini, calcolo mappa...
  -> Trovate 2913 cuciture da applicare.
  Rilascio memoria Blocco A...
  Applicazione correzioni a Blocco B...
  -> Blocco blocco_002-segmentato.parquet corretto e salvato.

--- Inizio cucitura: blocco_002-segmentato.parquet -> blocco_00

In [6]:
import pandas as pd
import glob
import os
import numpy as np
import gc # Importa il Garbage Collector

# --- 1. CONFIGURAZIONE ---
# ⭐️ Punta alla cartella finale cucita
INPUT_DIR = 'Dataset_Stitched_Finale' 
TIME_GAP_THRESHOLD = pd.Timedelta(hours=1)

print(f"--- 🔍 SCRIPT DI VERIFICA FINALE (Controllo ID) ---")
print(f"Lettura blocchi da: {INPUT_DIR}\n")

all_blocks = sorted(glob.glob(os.path.join(INPUT_DIR, '*.parquet')))

if not all_blocks:
    print(f"ERRORE: Nessun file blocco trovato in {INPUT_DIR}.")
else:
    total_missed_stitches = 0 # Contatore per le cuciture mancate

    print("--- Inizio scansione dei confini... ---")

    for i in range(len(all_blocks) - 1):
        path_A = all_blocks[i]
        path_B = all_blocks[i+1]
        
        try:
            df_A = pd.read_parquet(path_A)
            df_B = pd.read_parquet(path_B)

            # 1. Trova l'ULTIMO record di A (con il suo ID corretto)
            last_records_A = df_A.loc[df_A.groupby('MMSI')['Timestamp'].idxmax()]
            last_records_A = last_records_A[['MMSI', 'Timestamp', 'TrajectoryID']].rename(
                columns={'Timestamp': 'Last_Timestamp', 'TrajectoryID': 'ID_A'}
            )

            # 2. Trova il PRIMO record di B (con il suo ID (si spera) corretto)
            first_records_B = df_B.loc[df_B.groupby('MMSI')['Timestamp'].idxmin()]
            first_records_B = first_records_B[['MMSI', 'Timestamp', 'TrajectoryID']].rename(
                columns={'Timestamp': 'First_Timestamp', 'TrajectoryID': 'ID_B'}
            )

            # 3. Unisci i confini
            boundary_check = pd.merge(last_records_A, first_records_B, on='MMSI')

            # 4. Calcola il TimeDiff
            boundary_check['TimeDiff'] = boundary_check['First_Timestamp'] - boundary_check['Last_Timestamp']

            # 5. Trova solo i gap che DOVEVANO essere cuciti (<= 1 ora)
            stitchable_gaps = boundary_check[boundary_check['TimeDiff'] <= TIME_GAP_THRESHOLD]
            
            if not stitchable_gaps.empty:
                # 6. VERIFICA: Trova se qualcuno di questi ha ID diversi
                missed_stitches = stitchable_gaps[stitchable_gaps['ID_A'] != stitchable_gaps['ID_B']]
                
                local_missed_count = len(missed_stitches)
                total_missed_stitches += local_missed_count
                
                print(f"Confine {i+1}: Trovati {len(stitchable_gaps)} gap (<= 1h). Di questi, {local_missed_count} cuciture mancate.")
            else:
                print(f"Confine {i+1}: Nessun gap (<= 1h) trovato.")

        except Exception as e:
            print(f"  -> ERRORE durante il controllo del confine {i+1}: {e}")
            
        # Pulisci la memoria
        finally:
            del df_A, df_B, last_records_A, first_records_B, boundary_check, stitchable_gaps
            if 'missed_stitches' in locals(): del missed_stitches
            gc.collect()

    print("\n--- ✅ VERIFICA COMPLETATA ---")
    print(f"RISULTATO FINALE: Trovate {total_missed_stitches} cuciture mancate.")
    
    if total_missed_stitches == 0:
        print("🎉 CONGRATULAZIONI: Il dataset è perfettamente cucito!")
    else:
        print("⚠️ ATTENZIONE: Ci sono ancora traiettorie spezzate nel dataset.")

--- 🔍 SCRIPT DI VERIFICA FINALE (Controllo ID) ---
Lettura blocchi da: Dataset_Stitched_Finale

--- Inizio scansione dei confini... ---
Confine 1: Trovati 3127 gap (<= 1h). Di questi, 0 cuciture mancate.
Confine 2: Trovati 2913 gap (<= 1h). Di questi, 0 cuciture mancate.
Confine 3: Trovati 3666 gap (<= 1h). Di questi, 0 cuciture mancate.
Confine 4: Trovati 3450 gap (<= 1h). Di questi, 0 cuciture mancate.
Confine 5: Trovati 3353 gap (<= 1h). Di questi, 0 cuciture mancate.
Confine 6: Trovati 2838 gap (<= 1h). Di questi, 0 cuciture mancate.
Confine 7: Trovati 3095 gap (<= 1h). Di questi, 0 cuciture mancate.
Confine 8: Trovati 2501 gap (<= 1h). Di questi, 0 cuciture mancate.
Confine 9: Trovati 2540 gap (<= 1h). Di questi, 0 cuciture mancate.
Confine 10: Trovati 2657 gap (<= 1h). Di questi, 0 cuciture mancate.
Confine 11: Trovati 2115 gap (<= 1h). Di questi, 0 cuciture mancate.
Confine 12: Trovati 2019 gap (<= 1h). Di questi, 0 cuciture mancate.
Confine 13: Trovati 2353 gap (<= 1h). Di ques